# Chapter 6

[Chapter 6](https://learn.deeplearning.ai/courses/langchain/lesson/6/evaluation) was about evaluation, i.e. unit testing the implementation from chapter 5.

So let's see howe the Wittmann-Tours example works under the hood.

## Wittmann Tours basic setup

In [1]:
import os
import glob

def get_blog_post_files(path_to_blog):

    pattern = os.path.join(path_to_blog, "**/*.md")
    return sorted(glob.glob(pattern, recursive=True))

def get_blogpost(path_to_blogpost):
    with open(path_to_blogpost, 'r') as file:
        content = file.read()

    return content

In [2]:
blogpost_files = get_blog_post_files("./../wt-blogposts")
blogpost_files[0:5]

['./../wt-blogposts/3-tage-in-melbourne/index.md',
 './../wt-blogposts/addis-abeba-die-hauptstadt-athiopiens/index.md',
 './../wt-blogposts/aksum-aufbewahrungsort-der-bundeslade/index.md',
 './../wt-blogposts/am-fusse-des-cotopaxi/index.md',
 './../wt-blogposts/an-der-grenze-von-mexiko-nach-belize/index.md']

In [3]:
blogposts = [get_blogpost(path_to_blogpost) for path_to_blogpost in blogpost_files]

In [4]:
print(blogposts[0][317:835])

# 3 Tage in Melbourne

Auch wenn Canberra die offizielle Hauptstadt Australiens ist, so liefern sich Melbourne und Sydney als die beiden größten Städte des Kontinents ein Wettrennen um die Wahrnehmung als geistige Kapitale des Landes. Nach relativ viel Naturprogramm besuchten wir Melbourne, „[the world's most liveable city](https://www.smh.com.au/business/the-economy/melbourne-named-worlds-most-liveable-city-by-the-economist-for-seventh-year-20170816-gxx1kg.html)“, zu der sie der Economist wiederholt gekürt hat. 


In [5]:
from IPython.display import display, Markdown

def display_response(response):
    display(Markdown(response))

## Recreate Wittmann-Tours Mini-Rag

In [6]:
import os
from dotenv import load_dotenv

load_dotenv() #contains the OPENAI_API_KEY

True

Langchain is evolving fast, and below I ran into quite a few issues, but upgrading to the latest version fixed that.

In [7]:
#!pip install --upgrade langchain
#!pip install --upgrade langchain_openai
#!pip install --upgrade langchain_community


In [8]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Initialize the embedding model
embedding = OpenAIEmbeddings()

In [9]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

loaders = [UnstructuredMarkdownLoader(blogpost_file) for blogpost_file in blogpost_files]

index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embedding
).from_loaders(loaders)

/Users/chrwittm/miniforge3/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
query = "Wie hieß der Camping Platz unter den Koalas?"
response = index.query(query, llm = llm)

display_response(response)

Der Campingplatz hieß Bimbi Park.

## My QA-examples

In [11]:
examples = [
    {
        "query": "Wie hieß unser Guide im Masoala Regenwald?",
        "answer": "Armand"
    },
    {
        "query": "Wie hieß der Camping Platz unter den Koalas?",
        "answer": "Bimbi Park"
    }
]

## LLM-generated examples

In [12]:
from langchain.evaluation.qa import QAGenerateChain

example_gen_chain = QAGenerateChain.from_llm(llm)

Let's just create one example:

In [13]:
new_example = example_gen_chain.apply_and_parse([{"doc": blogposts[0]}])
print(new_example)

/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[{'qa_pairs': {'query': 'What are some of the key features that distinguish Melbourne from Sydney, as mentioned in the document "3 Tage in Melbourne"?', 'answer': 'Melbourne is distinguished from Sydney by its impressive buildings from the Gold Rush era, a rich art and culture scene, diverse shopping opportunities, and a variety of gastronomy. While Sydney is noted for its unique coastal location, Melbourne is characterized by its colonial architecture, such as the Flinders Street Station, and its European-style lanes and arcades. Additionally, Melbourne is perceived as less overwhelming due to its limited number of skyscrapers in the city center, contributing to a more intimate urban experience.'}}]


There are at least two major problems with this new question:

- It is in the wrong language: English questions will not be answered correctly, because the origin sources in German.
- The question is too broad because it talks about "the document", and it does not relate to the title. Therefore, during retrieval, it would be hard to find the original source.

To fix this, let's update the prompt template.

In [14]:
example_gen_chain.prompt

PromptTemplate(input_variables=['doc'], input_types={}, partial_variables={}, template='You are a teacher coming up with questions to ask on a quiz. \nGiven the following document, please generate a question and answer based on that document.\n\nExample Format:\n<Begin Document>\n...\n<End Document>\nQUESTION: question here\nANSWER: answer here\n\nThese questions should be detailed and be based explicitly on information in the document. Begin!\n\n<Begin Document>\n{doc}\n<End Document>')

Here is the original prompt template:

In [15]:
print(example_gen_chain.prompt.template)

You are a teacher coming up with questions to ask on a quiz. 
Given the following document, please generate a question and answer based on that document.

Example Format:
<Begin Document>
...
<End Document>
QUESTION: question here
ANSWER: answer here

These questions should be detailed and be based explicitly on information in the document. Begin!

<Begin Document>
{doc}
<End Document>


Here is the new prompt template, notice that the keywords are still in English: QUESTION, ANSWER

In [16]:
from langchain.prompts import PromptTemplate

german_prompt_template = """
Du bist ein Lehrer, der Quizfragen entwickelt. 
Basierend auf dem folgenden Blogbeitrag, erstelle bitte eine spezifische Frage und eine passende Antwort, die sich direkt auf den Inhalt dieses Blogbeitrags beziehen.

Achte darauf, dass die Frage zu einem bestimmten Thema des Blogbeitrags passt, damit sie sich auf eine genaue Stelle des Dokuments bezieht. Da wir ein Retrieval-Augmented-Generation-Modell testen, sollte die Frage so gestaltet sein, dass die Antwort aus dem Dokument eindeutig abgeleitet werden kann.

Beispielformat:
<Begin Dokument>
...
<End Dokument>
QUESTION: Frage hier
ANSWER: Antwort hier

Diese Fragen sollten detailliert sein und sich ausdrücklich auf Informationen im Dokument beziehen. Los geht's!

<Begin Dokument>
{doc}
<End Dokument>
"""

qa_prompt = PromptTemplate(template=german_prompt_template, input_variables=["doc"])



In [17]:
# Update the QAGenerateChain with the new prompt template
example_gen_chain.prompt = qa_prompt

# Genberate a new example
new_example = example_gen_chain.apply_and_parse([{"doc": blogposts[0]}])

print(new_example)

/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


[{'qa_pairs': {'query': 'Welche Sehenswürdigkeit in Melbourne wird als ältester Bahnhof der Stadt beschrieben und was ist ein auffälliges Merkmal dieses Bahnhofs?', 'answer': 'Die Flinders Street Station wird als Melbournes ältester Bahnhof beschrieben. Ein auffälliges Merkmal sind die vielen Uhren über dem Eingang, die nicht die Zeit in verschiedenen Städten der Welt anzeigen, sondern den Zeitpunkt der Abfahrt der nächsten Züge.'}}]


Besser :)

In [18]:
new_examples = []

# Loop over blogposts and apply the example_gen_chain one by one
for blogpost in blogposts[:5]:
    try:
        # Apply the chain to each blogpost
        example = example_gen_chain.apply_and_parse([{"doc": blogpost}])
        new_examples.append(example[0]["qa_pairs"])
    except Exception as e:
        print(f"Error processing blogpost: {e}")

# Print or display the generated examples
print(new_examples)


/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/chrwittm/miniforge3/lib/python3.10/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output p

[{'query': 'Welche Sehenswürdigkeit in Melbourne wird als ältester Bahnhof der Stadt beschrieben und was ist ein auffälliges Merkmal dieses Bahnhofs?', 'answer': 'Die Flinders Street Station wird als Melbournes ältester Bahnhof beschrieben. Ein auffälliges Merkmal sind die vielen Uhren über dem Eingang, die nicht die Zeit in verschiedenen Städten der Welt anzeigen, sondern den Zeitpunkt der Abfahrt der nächsten Züge.'}, {'query': 'Welche Bedeutung hatte der Eukalyptusbaum für die Entwicklung von Addis Abeba, und welche Probleme brachte seine Einführung mit sich?', 'answer': 'Der Eukalyptusbaum wurde in Addis Abeba eingeführt und angepflanzt, um das Problem des Mangels an Feuerholz zu lösen, da dessen Transportwege weit waren. Obwohl der Eukalyptusbaum den Bedarf an Brennstoff deckte, verdrängte er einheimische Pflanzen durch sein schnelles Wachstum und seinen großen Wasserbedarf.'}, {'query': 'Welche Rolle spielt die Kirche der Heiligen Maria von Zion in Aksum in der Geschichte der äth

In [19]:
new_examples

[{'query': 'Welche Sehenswürdigkeit in Melbourne wird als ältester Bahnhof der Stadt beschrieben und was ist ein auffälliges Merkmal dieses Bahnhofs?',
  'answer': 'Die Flinders Street Station wird als Melbournes ältester Bahnhof beschrieben. Ein auffälliges Merkmal sind die vielen Uhren über dem Eingang, die nicht die Zeit in verschiedenen Städten der Welt anzeigen, sondern den Zeitpunkt der Abfahrt der nächsten Züge.'},
 {'query': 'Welche Bedeutung hatte der Eukalyptusbaum für die Entwicklung von Addis Abeba, und welche Probleme brachte seine Einführung mit sich?',
  'answer': 'Der Eukalyptusbaum wurde in Addis Abeba eingeführt und angepflanzt, um das Problem des Mangels an Feuerholz zu lösen, da dessen Transportwege weit waren. Obwohl der Eukalyptusbaum den Bedarf an Brennstoff deckte, verdrängte er einheimische Pflanzen durch sein schnelles Wachstum und seinen großen Wasserbedarf.'},
 {'query': 'Welche Rolle spielt die Kirche der Heiligen Maria von Zion in Aksum in der Geschichte d

In [20]:
examples += new_examples

In [21]:
response = index.query(examples[0]["query"], llm=llm)
display_response(response)

Euer Guide im Masoala Regenwald hieß Armand.

## Manual Evaluation

In [22]:
import langchain
langchain.debug = True

response = index.query(examples[0]["query"], llm=llm)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Wie hieß unser Guide im Masoala Regenwald?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Wie hieß unser Guide im Masoala Regenwald?",
  "context": "title: 'Drei Tage im Masoala-Regenwald' description: \"\" published: 2019-07-14 redirect_from: - https://wittmann-tours.de/drei-tage-im-masoala-regenwald/ categories: \"Brookesia, Chamäleon, Lemur, Madagaskar, Madagaskar, Maki, Masoala, Regenwald, roter Vari, Taggecko, Umweltschutz, Vari, Wald, Wanderung\" hero: ./img/wp-content-uploads-2019-06-CW-20180820-105656-0464-1024x683.jpg\n\nDrei Tage im Masoala-Regenwald\n\nNach einer knapp 2-stündigen Bootsfahrt von Nosy Mangabe aus erreichten wir unser Ziel, die Masoala Forest Lodge. Wir landeten an einem Strand und gingen kaum 200 Met

## LLM assisted evaluation

In [23]:
langchain.debug = False

model_predictions = [ index.query(example["query"], llm=llm) for example in examples ]

In [24]:
# Create a list to hold the merged dictionaries
predictions = []

# Loop over both examples and predictions simultaneously
for example, model_prediction in zip(examples, model_predictions):
    # Create a new dictionary with query, answer, and result keys
    merged_dict = {
        "query": example['query'],
        "answer": example['answer'],
        "result": model_prediction
    }
    # Append the merged dictionary to the merged_list
    predictions.append(merged_dict)

# Print or return the merged list
print(predictions[0])


{'query': 'Wie hieß unser Guide im Masoala Regenwald?', 'answer': 'Armand', 'result': 'Euer Guide im Masoala Regenwald hieß Armand.'}


In [25]:
examples[0]

{'query': 'Wie hieß unser Guide im Masoala Regenwald?', 'answer': 'Armand'}

In [26]:
from langchain.evaluation.qa import QAEvalChain
eval_chain = QAEvalChain.from_llm(llm)

graded_outputs = eval_chain.evaluate(examples, predictions)

In [27]:
graded_outputs

[{'results': 'CORRECT'},
 {'results': 'CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: CORRECT'},
 {'results': 'GRADE: INCORRECT'},
 {'results': 'GRADE: CORRECT'}]

In [28]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['results'])
    print()

Example 0:
Question: Wie hieß unser Guide im Masoala Regenwald?
Real Answer: Armand
Predicted Answer: Euer Guide im Masoala Regenwald hieß Armand.
Predicted Grade: CORRECT

Example 1:
Question: Wie hieß der Camping Platz unter den Koalas?
Real Answer: Bimbi Park
Predicted Answer: Der Campingplatz hieß Bimbi Park.
Predicted Grade: CORRECT

Example 2:
Question: Welche Sehenswürdigkeit in Melbourne wird als ältester Bahnhof der Stadt beschrieben und was ist ein auffälliges Merkmal dieses Bahnhofs?
Real Answer: Die Flinders Street Station wird als Melbournes ältester Bahnhof beschrieben. Ein auffälliges Merkmal sind die vielen Uhren über dem Eingang, die nicht die Zeit in verschiedenen Städten der Welt anzeigen, sondern den Zeitpunkt der Abfahrt der nächsten Züge.
Predicted Answer: Die Flinders Street Station wird als der älteste Bahnhof in Melbourne beschrieben. Ein auffälliges Merkmal dieses Bahnhofs sind die vielen Uhren, die sich direkt über dem Eingang befinden und den Zeitpunkt der A

## LangSmith

LangChain Plus (as it was still called in the course) has become LangSmith.

Signing up at https://www.langchain.com/ -> Login via: https://smith.langchain.com/

Generated API-Key, added it to my .env file

Here is a [Getting started Guide](https://docs.smith.langchain.com/). Since this deviates from the current course, I save it for later to return to this topic.





## Wrapping up

This chapter was interesting, because I did not touch the subject before, but it resulted in more code than I had anticipated, especially because langchain is evolving quickly, and I ran into errors because of outdated packages.

Going beyond just small PoCs, get the impression that evaluation, tracing and debugging might be something where Langchain can shine.